In [1]:
import requests
from bs4 import BeautifulSoup
import os

# URL of the news article
url = 'https://news.detik.com/berita/d-6934646/wakil-ketua-dprd-depok-datangi-gbi-cinere-bellevue-yang-diprotes-warga?single=1'
# Send a GET request to the website
response = requests.get(url)

# Parse the website content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find the news content in the parsed HTML
# This depends on the structure of the website, but often news content is inside <p> tags
news_content = soup.find_all('p')

# Extract and print the text from each <p> tag
for p in news_content:
    print(p.get_text())


Wakil Ketua DPRD Kota Depok Hendrik Tangke Allo mendatangi lokasi ibadah jemaat GBI Cinere Bellevue di Gandul, Depok, Jawa Barat (Jabar). Hendrik menegaskan negara harus hadir memberikan rasa aman dan nyaman terhadap seluruh umat beragama.
"Pemerintah harus hadir memberikan rasa aman dan nyaman terhadap seluruh umat agama apapun agamanya untuk melakukan ibadahnya dengan baik, termasuk di Kapel ini yang di Cinere," ujar dia di lokasi, Sabtu (16/9/2023).
Hendrik kemudian mengaku mendengar kegiatan peribadatan akan dilakukan secara online atau live streaming, usai kapel GBI Cinere Bellevue yang terletak di ruko 3 lanta itu didatangi puluhan warga. Hendrik menyesalkan hal tersebut.

ADVERTISEMENT


SCROLL TO CONTINUE WITH CONTENT

"Saya dengar Pemkot Depok, melalui Kesbangpol, sudah melakukan rapat beberapa waktu lalu. Dan diputuskan kegiatan ibadah di sini dihentikan untuk sementara. Nah ini yang saya sangat sesalkan, hanya online bisa," ucap Hendrik.
Pada kesempatan yang sama, pihak GBI 

In [2]:
# Open the output file in write mode
with open('C:/Users/ahini/Downloads/projek_pi_4/scorpus_pi/output1.txt', 'w', encoding='utf-8') as f:
    # Extract and write the text from each <p> tag to the file
    for p in news_content:
        f.write(p.get_text() + '\n')

In [7]:
import requests
from bs4 import BeautifulSoup
import os

# Daftar URL untuk di-scrape
urls = [
    "https://www.kompasiana.com/dian96997/64cbdf684addee3feb2fbbc2/komunikasi-tanpa-kata-posisi-budaya-visual-dalam-ilmu-komunikasi",
    "https://www.kompasiana.com/rizqiakbari4638/64a19d794addee41f4148982/iklan-dan-periklanan-di-era-digital",
    "https://www.kompasiana.com/pumacommunicationpresuniv0608/648d5f7208a8b55c7d7bc132/mahasiswa-ilmu-komunikasi-president-university-lakukan-kunjungan-ke-starvision",
    "https://www.kompasiana.com/kartiah367/6481fd4829f19e62881fc272/merangkul-keunikan-diri-memperkuat-self-esteem-melalui-komunikasi-antar-pribadi-yang-positif",
    "https://www.kompasiana.com/ekameiliyani/642fe6a44addee2b8f21b652/menjadi-pengusaha-muda-adalah-investasi-yang-tepat-untuk-masa-depan",
    "https://www.kompasiana.com/zainul16921/6504d2934addee305e68e6a2/transformasi-bisnis-langkah-menuju-sukses",
    "https://www.kompasiana.com/fitriawardani8538/65041b75fdf1c501c67dd972/upgrade-jaringan-komunikasi-modernisasi-media-komunikasi",
    "https://www.kompasiana.com/fitriawardani8538/650416cd08a8b515af672692/modernisasi-teknologi-mengubah-cara-berkomunikasi",
    "https://www.kompasiana.com/fitriawardani8538/6503941be3b78f636b59c9a2/perubahan-interaksi-sosial-online-modernisasi-dalam-komunikasi",
    "https://www.kompasiana.com/fitriawardani8538/65038f31e3b78f5fb61b3942/keamanan-digital-di-era-teknologi-modernisasi-teknologi-komunikasi",
    "https://www.kompasiana.com/muhamadqli/6501ca3a4addee175b290432/esperanto-bahasa-internasional-untuk-komunikasi-global",
    "https://www.kompasiana.com/fitriawardani8538/6502414f4addee11df477ce2/modernisasi-teknologi-chat-dan-pesan-singkat",
    "https://www.kompasiana.com/fitriawardani8538/6501c6e808a8b535a0453712/modernisasi-teknologi-perkembangan-media-sosial",
    "https://www.kompasiana.com/fitriawardani8538/6500f59f08a8b569c556b6c2/jaringan-kecepatan-tinggi-modernisasi-media-komunikasi",
    "https://www.kompasiana.com/fitriawardani8538/6500f331e1a1674c3c2a97e2/modernisasi-teknologi-tantangan-komunikasi-antarbudaya",
    "https://www.kompasiana.com/fitriawardani8538/6500f0b54addee57a914fb73/perubahan-pola-komunikasi-modernisasi-dalam-komunikasi",
    "https://www.kompasiana.com/syahrialsyahrial0585/64fd8f7e4addee55c96e9f82/melacak-jejak-mahasiswa-do-ketidakaktifan-komunikasi-dan-solusi",
    "https://www.kompasiana.com/fitriawardani8538/64cae0b108a8b503b77fd992/komunikasi-dalam-proses-negosiasi-mencapai-win-win-solution",
    "https://www.kompasiana.com/fitriawardani8538/64c50edd4addee302b6d3832/komunikasi-dan-teknik-negosiasi-yang-sukses-dalam-berbisnis",
    "https://www.kompasiana.com/fitriawardani8538/64c433bb08a8b57901143842/peran-komunikasi-dalam-pengembangan-dan-pemeliharaan-hubungan-pelanggan",
    "https://www.kompasiana.com/muhammadazkia11/64ad772a4addee3a2a41deb3/prinsip-prinsip-komunikasi-islam-dalam-dunia-bisnis",
    "https://www.kompasiana.com/dewilailypurnamasari/642acdb74addee6fec20d7c4/upgrade-skill-mahasiswa-kelas-komunikasi-dengan-jadi-kompasianer",
    "https://www.kompasiana.com/srirahmawati1809/63a6fccb906beb0c191ff8d2/keterampilan-komunilasi-menjadi-hal-utama-dalam-bisnis",
    "https://www.kompasiana.com/nurulhanifah/639ac3044addee2e4e04ba52/teknik-teknik-lobying-dalam-dunia-bisnis",
    "https://www.kompasiana.com/dewiani63893/63976efa29f19e1dde6c3ba2/pentingnya-komunikasi-bisnis-untuk-keberhasilan-suatu-usaha"
]

for url in urls:
    # Kirim permintaan GET
    response = requests.get(url)

    # Parse konten dengan BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Ekstrak judul halaman sebagai nama file (Anda mungkin perlu menyesuaikan ini tergantung pada struktur halaman web)
    title = soup.title.string

    # Buat versi singkat dari judul untuk digunakan sebagai nama file
    short_title = title[:50]  # Ambil 50 karakter pertama dari judul

    # Ganti karakter yang tidak valid dalam nama file
    filename = "".join(c for c in short_title if c.isalnum() or c.isspace()).rstrip() + ".txt"

    # Ekstrak teks dari halaman web
    text = soup.get_text()

    # Simpan teks ke file .txt
    with open(filename, 'w') as f:
        f.write(text)


KeyboardInterrupt: 

In [3]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.mitrakeluarga.com/artikel/gejala-depresi'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find all script tags
scripts = soup.find_all('script')

for script in scripts:
    # Print the content of each script tag
    print(script.string)
    
# Ekstrak judul halaman sebagai nama file (Anda mungkin perlu menyesuaikan ini tergantung pada struktur halaman web)
    title = soup.title.string

    # Buat versi singkat dari judul untuk digunakan sebagai nama file
    short_title = title[:50]  # Ambil 50 karakter pertama dari judul

    # Ganti karakter yang tidak valid dalam nama file
    filename = "".join(c for c in short_title if c.isalnum() or c.isspace()).rstrip() + ".txt"

    # Ekstrak teks dari halaman web
    text = soup.get_text()

    # Simpan teks ke file .txt
    with open(filename, 'w') as f:
        f.write(text)


if(!window._gtm_init){window._gtm_init=1;(function(w,n,d,m,e,p){w[d]=(w[d]==1||n[d]=='yes'||n[d]==1||n[m]==1||(w[e]&&w[e][p]&&w[e][p]()))?1:0})(window,navigator,'doNotTrack','msDoNotTrack','external','msTrackingProtectionEnabled');(function(w,d,s,l,x,y){w[x]={};w._gtm_inject=function(i){if(w.doNotTrack||w[x][i])return;w[x][i]=1;w[l]=w[l]||[];w[l].push({'gtm.start':new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],j=d.createElement(s);j.async=true;j.src='https://www.googletagmanager.com/gtm.js?id='+i;f.parentNode.insertBefore(j,f);};w[y]('GTM-PPXPXZ4')})(window,document,'script','dataLayer','_gtm_ids','_gtm_inject')}
(function(){var l=document.createElement('link');l.rel="stylesheet";l.href="https://fonts.googleapis.com/css2?family=Poppins:wght@400;500;600;700&display=swap";document.querySelector("head").appendChild(l);})();
window.__NUXT__=(function(a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r){return {layout:"default",data:[{article:{type:e,id:"81",author:f,title:m,slug:

In [6]:
import requests
from bs4 import BeautifulSoup

# URL artikel
url = "https://www.kompasiana.com/rizqiakbari4638/64a19d794addee41f4148982/iklan-dan-periklanan-di-era-digital"

# Kirim permintaan GET
response = requests.get(url)

# Parse konten dengan BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Temukan elemen dengan atribut itemprop="articleBody"
article_body = soup.find(itemprop='articleBody')

# Dapatkan teks dari setiap elemen <p> dan simpan ke dalam list
paragraphs = [p.get_text() for p in article_body.find_all('p')]

#Gabungkan list menjadi satu string, dengan setiap paragraf dipisahkan oleh baris baru
text = '\n'.join(paragraphs)

print(text)


# Ekstrak judul halaman sebagai nama file (Anda mungkin perlu menyesuaikan ini tergantung pada struktur halaman web)
title = soup.title.string

# Buat versi singkat dari judul untuk digunakan sebagai nama file
short_title = title[:50]  # Ambil 50 karakter pertama dari judul

# Ganti karakter yang tidak valid dalam nama file
filename = "".join(c for c in short_title if c.isalnum() or c.isspace()).rstrip() + ".txt"


# Simpan teks ke file .txt
with open(filename, 'w') as f:
    f.write(text)



Di era digital yang semakin maju, periklanan telah menjadi salah satu faktor paling penting dalam dunia bisnis. Perkembangan teknologi informasi dan komunikasi telah mengubah cara kita berinteraksi dengan merek dan produk. Dengan begitu banyak orang yang terhubung secara online, periklanan di era digital telah menjadi modal utama untuk mencapai konsumen potensial. Dalam artikel ini, kita akan membahas pentingnya periklanan di era digital dan mengapa strategi pemasaran digital yang efektif menjadi kunci keberhasilan dalam memenangkan persaingan bisnis.
Jangkauan yang Luas dan Targetisasi yang Akurat
Salah satu keuntungan utama periklanan di era digital adalah jangkauan yang luas dan targetisasi yang akurat. Dalam dunia digital, bisnis dapat mencapai audiens global dengan biaya yang relatif rendah dibandingkan dengan media tradisional. Platform periklanan digital seperti mesin pencari, media sosial, dan situs web dengan lalu lintas tinggi memungkinkan perusahaan untuk menjangkau konsumen